In [37]:
# Used to use split in map
from code import compile_command
from operator import methodcaller
from typing import Tuple

# Used to access content of URL
import requests

# json package
import json

# Used to get uniform random sample
from numpy import random as rd

# How large is the sample?
# Extracted with: num_lines = sum(1 for line in open('ticker.txt'))

total_companies = 12175
number_of_samples = 50

# Pick random sample uniform over [1,total_companies]
samples = rd.randint(0,total_companies,number_of_samples)

# Store list of companies as a list with elements of the form ['name','identifier']
lines_list_raw = map(str.strip, open('ticker.txt').readlines())
lines_list = list(map(methodcaller('split', '\t'), lines_list_raw))

# Generate list of identifies to read
id_samples = [ lines_list[i][1] for i in samples]


import time 

def bad_ids() -> set:
    # Trying to automate finding bad IDs. Unfinished.
    # Many entities are marked as 'other'. Do not want to deal with these ids.

    bad_ids = set()

    for x in id_samples:
        s = x[1]
        link = 'https://data.sec.gov/submissions/CIK{}.json'.format(s.zfill(10))
        hdr = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36'}
        
        response = requests.get(link,headers=hdr)
        time.sleep(0.01) 
        byte_content = response.content
        string_content = byte_content.decode("utf-8")
        json_content = json.loads(string_content)
        

        if json_content['entityType'] == 'other':
            bad_ids.add(s.zfill(10))
    
    return bad_ids



def get_data(my_list : list[str], data_base : list) -> list[Tuple]:

    bad_samples = ['0001807689','0001495584','0001093428','0000893739','0000890725','0001504619']
    # So far must be manually updated...

    for id in my_list:
        if id not in bad_samples:
                
            print('Company id is: ' + id.zfill(10))
            
            link = 'https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json'.format(id.zfill(10))
            link_test = 'https://data.sec.gov/submissions/CIK{}.json'.format(id.zfill(10))

            hdr = {'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36'}
            
            response = requests.get(link,headers=hdr)
            response_test = requests.get(link_test,headers=hdr)

            # Content is now of type byte
            byte_content = response.content
            byte_content_test = response_test.content

            # Content is now of type string
            string_content = byte_content.decode("utf-8")
            string_content_test = byte_content_test.decode("utf-8")

            # Content is now a json 
            json_content_test = json.loads(string_content_test)

            if not json_content_test['entityType'] == 'other':
                json_content = json.loads(string_content)

                if 'entityName' in json_content:
                    # print('Getting information on ' + json_content['entityName'])
                    if 'facts' in json_content:
                        if 'us-gaap' in json_content['facts']:
                            if 'CommonStockSharesOutstanding' in json_content['facts']['us-gaap']:
                                shares = json_content['facts']['us-gaap']['CommonStockSharesOutstanding']['units']['shares'][-1]['val']
                                # print('CommonStockSharesOutstanding is ' + str(shares))
                                data_base.append((shares,json_content['entityName']))
                            elif 'PreferredStockSharesOutstanding' in json_content['facts']['us-gaap']:
                                shares = json_content['facts']['us-gaap']['PreferredStockSharesOutstanding']['units']['shares'][-1]['val']
                                # print('PreferredStockSharesOutstanding is ' + str(shares))
                                data_base.append((shares,json_content['entityName']))
    return sorted(data_base)

data_base = []

get_data(id_samples,data_base)


 


Company id is: 0001234006
Company id is: 0000354647
Company id is: 0001856242
Company id is: 0000013156
Company id is: 0001796514
Company id is: 0001843121
Company id is: 0001383312
Company id is: 0000911216
Company id is: 0001817740
Company id is: 0001643918
Company id is: 0001532346
Company id is: 0001862150
Company id is: 0001113232
Company id is: 0001525852
Company id is: 0001543418
Company id is: 0001803901
Company id is: 0001085913
Company id is: 0001024672
Company id is: 0001420520
Company id is: 0000019871
Company id is: 0001358356
Company id is: 0001530249
Company id is: 0001610315
Company id is: 0001114446
Company id is: 0001852061
Company id is: 0001830180
Company id is: 0000276720
Company id is: 0001834974
Company id is: 0001690947
Company id is: 0001722926
Company id is: 0001416090
Company id is: 0001428389
Company id is: 0001747068
Company id is: 0001050797
Company id is: 0001844507
Company id is: 0000096699
Company id is: 0001334740
Company id is: 0001818876
Company id i

[(0, 'Blockchain Coinvestors Acquisition Corp. I'),
 (0, 'C5 Acquisition Corporation'),
 (0, 'GENERATION ASIA I ACQUISITION LIMITED'),
 (0, 'MAGNUM OPUS ACQUISITION LIMITED'),
 (0, 'Social Capital Hedosophia Holdings Corp. IV'),
 (777190, 'FARMERS & MERCHANTS BANCORP'),
 (966132, 'CHICAGO RIVET & MACHINE CO'),
 (1854403, 'TECHNICAL COMMUNICATIONS CORP'),
 (2500000, 'ACHARI VENTURES HOLDINGS CORP. I'),
 (7726232, 'FS BANCORP,\xa0INC.'),
 (11309412, 'Cingulate Inc.'),
 (14890514, 'LIMESTONE BANCORP, INC.'),
 (23457000, 'ATOMERA INCORPORATED'),
 (23980645, 'PURE CYCLE CORPORATION'),
 (24116083, 'Galaxy Gaming, Inc.'),
 (25451125, 'METROCITY BANKSHARES,\xa0INC.'),
 (26644252, 'TARSUS PHARMACEUTICALS, INC.'),
 (33064000, 'AXCELIS TECHNOLOGIES INC'),
 (37640008, 'MARKETAXESS HOLDINGS INC.'),
 (39136473, 'GLADSTONE COMMERCIAL CORP'),
 (65501000, 'COLUMBIA SPORTSWEAR COMPANY'),
 (83303000, 'iStar\xa0Inc.'),
 (117300000, 'BROADRIDGE FINANCIAL SOLUTIONS, INC.'),
 (120111678, 'ENJOY TECHNOLOGY, I